In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install sentence-transformers faiss-cpu pillow numpy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 112.4 MB/s eta 0:00:00


In [3]:
import os
import json
import torch
import requests
import faiss
import numpy as np
from PIL import Image
from sentence_transformers import SentenceTransformer
from collections import defaultdict

In [4]:
CATEGORY_MAP = ["상의", "바지", "아우터", "신발", "가방"]

In [5]:
from sentence_transformers import SentenceTransformer
import torch

def load_model():
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"➡️ DEVICE: {DEVICE}")

    text_model = SentenceTransformer(
        "upskyy/bge-m3-korean",
        device=DEVICE
    )
    text_model.eval()
    print("🤖 Text model (bge-m3) loaded successfully!")

    return text_model


def _flush_batch(text_model, texts, metas, cats, buffers):
    text_vecs = text_model.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
    )

    for tv, m, c in zip(text_vecs, metas, cats):
        buffers[c].append({
            "vector": tv,
            "metadata": m,
        })

def load_all_items(jsonl_dir):
    items = []

    for fname in os.listdir(jsonl_dir):
        if not fname.endswith(".jsonl"):
            continue

        path = os.path.join(jsonl_dir, fname)
        print(f"📂 Loading {path}")

        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                items.append(json.loads(line))

    print(f"✅ 총 {len(items)}개 아이템 로드 완료\n")
    return items

In [6]:
def build_style_db(
    jsonl_dir,
    output_root: str,
    batch_size: int = 32,
):
    os.makedirs(output_root, exist_ok=True)

    text_model = load_model()
    items = load_all_items(jsonl_dir)

    buffers = defaultdict(list)

    text_batch, meta_batch, cat_batch = [], [], []

    print(f"🔗 총 {len(items)}개 아이템 처리 시작\n")

    for idx, item in enumerate(items):
        print(f"➡️ {idx+1}/{len(items)} 처리 중")

        main_cat = item.get("main_cat_name")
        if main_cat not in CATEGORY_MAP:
            continue

        # ---------------------------
        # 🔥 임베딩 텍스트: tpo, mood
        # ---------------------------
        tpo = item.get("tpo")
        mood = item.get("mood")
        text_input = tpo + ", " + mood
        print(f"text_input: {text_input}")

        # ---------------------------
        # 🔥 metadata (네가 준 스키마 그대로)
        # ---------------------------

        metadata = {
            "product_id": item.get("product_id"),
            "brand": item.get("brand"),
            "gender": item.get("gender"),
            "price": item.get("price"),
            "price_raw": item.get("price_raw"),
            "main_cat_name": item.get("main_cat_name"),
            "sub_cat_name": item.get("sub_cat_name"),
            "style": item.get("style_name"),
            "texture": item.get("texture_name"),
            "pattern": item.get("pattern_name"),
            "fit": item.get("fit_name"),
            "mood": item.get("mood"),
            "tpo": item.get("tpo"),
            "color": item.get("색상"),
            "img_url": item.get("img_url"),
            "description": item.get("description")
        }

        text_batch.append(text_input)
        meta_batch.append(metadata)
        cat_batch.append(main_cat)

        # ---------- batch encode ----------
        if len(text_batch) == batch_size:
            _flush_batch(
                text_model,
                text_batch,
                meta_batch,
                cat_batch,
                buffers,
            )
            text_batch, meta_batch, cat_batch = [], [], []

    # leftover
    if text_batch:
        _flush_batch(
            text_model,
            text_batch,
            meta_batch,
            cat_batch,
            buffers,
        )

    # ---------- FAISS 저장 ----------
    for cat_key, entries in buffers.items():
        print(f"\n💾 Saving category: {cat_key} ({len(entries)} items)")

        vectors = np.array([e["vector"] for e in entries]).astype("float32")
        metas = [e["metadata"] for e in entries]

        index = faiss.IndexFlatIP(vectors.shape[1])
        index.add(vectors)

        save_dir = os.path.join(output_root, cat_key)
        os.makedirs(save_dir, exist_ok=True)

        faiss.write_index(index, f"{save_dir}/index.faiss")

        with open(f"{save_dir}/metadata.jsonl", "w", encoding="utf-8") as f:
            for m in metas:
                f.write(json.dumps(m, ensure_ascii=False) + "\n")

        print(f"✅ {cat_key} 완료")

    print("\n🎉 TPO text-only DB 구축 완료!")


In [7]:
def main():
    jsonl_dir = "/content/drive/MyDrive/프메 10팀/final_db_color"
    output_root = "/content/drive/MyDrive/프메 10팀/민서/faiss/tpo"
    build_style_db(jsonl_dir=jsonl_dir, output_root=output_root,batch_size=32)

if __name__ == "__main__":
    main()

➡️ DEVICE: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

🤖 Text model (bge-m3) loaded successfully!
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/여_가방_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/여_신발_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/여_아우터_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/여_바지_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/여_상의_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/남_상의_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/남_가방_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/남_바지_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/남_신발_color.jsonl
📂 Loading /content/drive/MyDrive/프메 10팀/final_db_color/남_아우터_color.jsonl
✅ 총 1217개 아이템 로드 완료

🔗 총 1217개 아이템 처리 시작

➡️ 1/1217 처리 중
text_input: 남친룩/여친룩, 소개팅, 영화관, 애인과 데이트, 데이트
➡️ 2/1217 처리 중
text_input: 남친룩/여친룩, 소개팅, 영화관, 애인과

In [8]:
# 확인

import faiss
import json
import os
import numpy as np

def verify_category_db(category_path):
    print(f"🔍 검증 시작: {category_path}")

    index_path = os.path.join(category_path, "index.faiss")
    meta_path = os.path.join(category_path, "metadata.jsonl")

    if not os.path.exists(index_path) or not os.path.exists(meta_path):
        print("❌ 에러: .faiss 또는 .jsonl 파일이 존재하지 않습니다.")
        return

    # 1. FAISS 인덱스 로드
    index = faiss.read_index(index_path)
    total_vectors = index.ntotal

    # 2. 메타데이터 로드
    metas = []
    with open(meta_path, "r", encoding="utf-8") as f:
        for line in f:
            metas.append(json.loads(line))
    total_metas = len(metas)

    # 3. 개수 일치 확인
    print(f"📊 총 벡터 수: {total_vectors}")
    print(f"📊 총 메타데이터 수: {total_metas}")

    if total_vectors == total_metas:
        print("✅ [성공] 벡터와 메타데이터의 개수가 일치합니다.")
    else:
        print(f"❌ [실패] 개수 불일치! (차이: {abs(total_vectors - total_metas)})")

    # 4. 데이터 샘플링 확인 (product_id 확인)
    if total_metas > 0:
        print("\n--- 상위 3개 데이터 샘플 ---")
        for i in range(min(3, total_metas)):
            p_id = metas[i].get("product_id", "N/A")
            print(f"Index [{i}]: ID={p_id}")

    return total_vectors == total_metas

# 실행 예시 (아우터 카테고리 확인 시)
verify_category_db("/content/drive/MyDrive/프메 10팀/민서/faiss/tpo/신발")

🔍 검증 시작: /content/drive/MyDrive/프메 10팀/민서/faiss/tpo/신발
📊 총 벡터 수: 155
📊 총 메타데이터 수: 155
✅ [성공] 벡터와 메타데이터의 개수가 일치합니다.

--- 상위 3개 데이터 샘플 ---
Index [0]: ID=5627850
Index [1]: ID=5627844
Index [2]: ID=4633774


True